## Extended Maximum Likelihood fit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from iminuit import Minuit
from iminuit.cost import ExtendedUnbinnedNLL
#from distributions_nbs import g_s_nb, h_s_nb, g_b_nb, h_b_nb, g_s_nb_, h_s_nb_, g_b_nb_, h_b_nb_
from scrapbooks.distributions import *
from numba import njit, float64


In [102]:
#Load the data
data = np.loadtxt('data_storage/joint_pdf_sample_s100.csv', delimiter=',', skiprows=1)
#data = np.vstack((x_samples, y_samples)).T
#N = len(x_samples)
X, Y = data
N = len(X)

In [103]:
#Create the model
def model(X, Y, mu, sigma, beta, m, f, lmbda, mu_b, sigma_b):
    signal = f * g_s(X, beta, m, mu, sigma, 5.0, 0.0) * h_s(Y, 10.0, 0.0, lmbda)
    background = (1-f) * g_b(X, 5.0, 0.0) * h_b(Y, mu_b, sigma_b, 10.0, 0.0)
    return signal + background
    
def density(data, N, mu, sigma, beta, m, f, lmbda, mu_b, sigma_b):
    X, Y = data
    return N, N * model (X, Y, mu, sigma, beta, m, f, lmbda, mu_b, sigma_b)

In [104]:
#Create the model
def model_density(data, N, mu, sigma, beta, m, f, lmbda, mu_b, sigma_b):

    X, Y = data

    X_ulim = 5.0
    X_llim = 0.0
    Y_ulim = 10.0
    Y_llim = 0.0

    g_s = sps.crystalball.pdf(X, beta, m, mu, sigma) / (sps.crystalball.cdf(X_ulim, beta, m, mu, sigma) - sps.crystalball.cdf(X_llim, beta, m, mu, sigma))
    
    trunc_b = (Y_ulim - Y_llim)*lmbda
    h_s = sps.truncexpon.pdf(Y, trunc_b, Y_llim, 1/lmbda)

    g_b = sps.uniform.pdf(X, X_llim, X_ulim)

    a = (Y_llim - mu_b)/sigma_b
    b = (Y_ulim - mu_b)/sigma_b
    h_b = sps.truncnorm.pdf(Y, a, b, mu_b, sigma_b)

    signal = f * g_s * h_s
    background = (1-f) * g_b * h_b

    total = signal + background

    return N, N*total


In [105]:
#make the cost function which is the negative log likelihood
nll = ExtendedUnbinnedNLL((X,Y), model_density)

#Define the true parameters as starting values
N_ = 100000
mu_ = 3.0
sigma_ = 0.3
beta_ = 1.0
m_ = 1.4
f_ = 0.6
lmbda_ = 0.3
mu_b_ = 0.0
sigma_b_ = 2.5

mi = Minuit(nll, 
            N=N_,          # name=value format
            mu=mu_,
            sigma=sigma_,
            beta=beta_,
            m=m_,
            f=f_,
            lmbda=lmbda_,
            mu_b=mu_b_,
            sigma_b=sigma_b_)

#beta must be greater than 0
#m must be greater than 1

mi.limits["N"] = (0, None)
# mi.limits["mu"] = (0, None)
# mi.limits["sigma"] = (0, None)
mi.limits["beta"] = (0, None)
mi.limits["m"] = (1, None)
# mi.limits["lmbda"] = (0, None)
# mi.limits["mu_b"] = (-5, 5)
# mi.limits["sigma_b"] = (0, None)
# mi.limits["f"] = (0, 1)

#mi.scan()
#mi.strategy = 1
mi.migrad()
mi.hesse()
#mi.minos()


┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = -1.462e+06                 │              Nfcn = 301              │
│ EDM = 0.000122 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬─────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name    │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼─────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ N       │ 100.00e3  │  0.32e3   │            │            │    0    │         │       │
│ 1 │ mu      │  3.0072   │  0.0025   │            │            │         │         │       │
│ 2 │ sigma   │  0.2960   │  0.0024   │            │            │         │         │       │
│ 3 │ beta    │   0.971   │   0.021   │            │            │    0    │         │       │
│ 4 │ m       │   1.48    │   0.07    │            │            │    1    │         │       │
│ 5 │ f       │   0.601   │   0.004   │            │            │         │         │       │
│ 6 │ lmbda   │  0.2996   │  0.0021   │            │            │         │         │       │
│ 7 │ mu_b    │   0.02    │   0.08    │            │            │         │         │       │
│ 8 │ sigma_b │   2.48    │   0.04    │            │            │         │         │       │
└───┴─────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────────┬───────────────────────────────────────────────────────────────────────────────────────────┐
│         │         N        mu     sigma      beta         m         f     lmbda      mu_b   sigma_b │
├─────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│       N │  1.01e+05     19e-6    -27e-6   -0.5e-3     0.002 -0.039e-3    -11e-6    -0.000    0.0001 │
│      mu │     19e-6  6.42e-06     -3e-6    -27e-6     51e-6     -0e-6     -0e-6     -2e-6      1e-6 │
│   sigma │    -27e-6     -3e-6  5.58e-06     22e-6    -46e-6      2e-6      0e-6     14e-6     -8e-6 │
│    beta │   -0.5e-3    -27e-6     22e-6  0.000449   -1.2e-3  0.008e-3      1e-6    0.1e-3        -0 │
│       m │     0.002     51e-6    -46e-6   -1.2e-3    0.0043 -0.088e-3    -13e-6    -0.001    0.0003 │
│       f │ -0.039e-3     -0e-6      2e-6  0.008e-3 -0.088e-3  1.26e-05      2e-6  0.047e-3 -0.028e-3 │
│   lmbda │    -11e-6     -0e-6      0e-6      1e-6    -13e-6      2e-6  4.22e-06     11e-6      3e-6 │
│    mu_b │    -0.000     -2e-6     14e-6    0.1e-3    -0.001  0.047e-3     11e-6   0.00593   -0.0026 │
│ sigma_b │    0.0001      1e-6     -8e-6        -0    0.0003 -0.028e-3      3e-6   -0.0026   0.00129 │
└─────────┴───────────────────────────────────────────────────────────────────────────────────────────┘